In [1]:
!pip install category_encoders
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn as sk
from sklearn.preprocessing import StandardScaler
from category_encoders import TargetEncoder
from sklearn.preprocessing import OneHotEncoder 
from datetime import datetime
import seaborn as sns
from sklearn import linear_model
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import PolynomialFeatures
import pickle
from sklearn import metrics
import time
from sklearn.preprocessing import LabelEncoder

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 80 kB 760 kB/s 


/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
def dumppers(string1,string2):
  file = open(string1, 'wb')

  # dump information to that file
  pickle.dump(string2, file)

  # close the file
  file.close()

In [3]:
data3 = pd.read_csv("movies-revenue.csv")
data1 = pd.read_csv("movie-director.csv") #name
data2 = pd.read_csv("movie-voice-actors.csv") #movie


In [4]:
data1.rename(columns={'name': 'movie_title'}, inplace=True)
data2.rename(columns={'movie': 'movie_title'}, inplace=True)

In [5]:
data = pd.merge(data3,data1[['movie_title','director']],on='movie_title', how='left')

In [6]:
data_actors= data2.groupby('movie_title')['voice-actor'].agg(list)

In [7]:
my_data = pd.merge(data,data_actors,on='movie_title', how='left')

In [8]:
my_data.isnull().sum()

movie_title       0
release_date      0
genre            14
MPAA_rating      47
revenue           0
director        428
voice-actor     424
dtype: int64

In [9]:
my_data['release_date'] = pd.to_datetime(my_data['release_date'])
my_data['release_date'] = pd.DatetimeIndex(my_data['release_date']).year

In [10]:
!pip install tmdbv3api
from tmdbv3api import TMDb
tmdb = TMDb()
tmdb.api_key = 'f28265b95227d640ac243fdb15a509a6'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [11]:
tmdb.language = 'en'
tmdb.debug = True

In [12]:
from tmdbv3api import Movie
movie = Movie()

In [13]:
for index, row in my_data.iterrows():
  if(row["release_date"] > 2022):
    my_data.at[index,"release_date"] = row["release_date"] - 100

In [14]:
cnt = 0
for index, row in my_data.iterrows():
  if pd.isna(my_data.iloc[index,5]):
    search = movie.search(row["movie_title"])
    for res in search:
      #if res.original_title == row["movie_title"]:
      if "release_date" in res.keys():
        if str(row["release_date"]) in res.release_date:
          crew = movie.credits(res.id)["crew"]
          for x in crew:
            if(x["job"].lower() == "director"):
              cnt = cnt + 1
              my_data.at[index,"director"] = x.name
              #df.loc[len(df.index)] = [res.original_title, x.name] 
             # print(res.original_title+ ",  "+ x.name)
              break;
          break;
      
#print(cnt)

In [15]:
my_data.isnull().sum()

movie_title       0
release_date      0
genre            14
MPAA_rating      47
revenue           0
director         23
voice-actor     424
dtype: int64

In [16]:
fq = my_data.groupby('director').size()

In [17]:
fq

director
Adam Bernstein         1
Adam Shankman          3
Alan J. Pakula         1
Alan Parker            1
Alan Taylor            1
                      ..
William Brent Bell     1
William Dear           1
William Nicholson      1
Wolfgang Reitherman    5
full credits           1
Length: 317, dtype: int64

In [18]:
my_data

,movie_title,release_date,genre,MPAA_rating,revenue,director,voice-actor
0,Recess: School's Out,2001,Comedy,G,"$54,656,124",Chuck Sheetz,NaN
1,D2: The Mighty Ducks,1994,Comedy,PG,"$94,226,333",Sam Weisman,NaN
2,Home on the Range,2004,Comedy,PG,"$67,910,166",Will Finn,"[Randy Quaid, Estelle Harris, Cuba Gooding Jr...."
3,Young Black Stallion,2003,Adventure,G,"$9,254,344",Simon Wincer,NaN
4,What's Love Got to Do With It,1993,Drama,R,"$79,618,610",Brian Gibson,NaN
...,...,...,...,...,...,...,...
458,Sweet Home Alabama,2002,Comedy,PG-13,"$184,521,151",Andy Tennant,NaN
459,Play it to the Bone,1999,Comedy,R,"$13,180,524",Ron Shelton,NaN
460,The Odd Life of Timothy Green,2012,Drama,PG,"$54,914,942",Peter Hedges,NaN
461,Honey I Blew Up the Kid,1992,Comedy,PG,"$119,162,518",Randal Kleiser,NaN


In [19]:
for idx,row in my_data.iterrows():
  if pd.isna(my_data.iloc[idx,5]):
    continue
  my_data.at[idx,"director"]=fq[ my_data.at[idx,"director"]]

In [20]:
my_data

,movie_title,release_date,genre,MPAA_rating,revenue,director,voice-actor
0,Recess: School's Out,2001,Comedy,G,"$54,656,124",1,NaN
1,D2: The Mighty Ducks,1994,Comedy,PG,"$94,226,333",2,NaN
2,Home on the Range,2004,Comedy,PG,"$67,910,166",1,"[Randy Quaid, Estelle Harris, Cuba Gooding Jr...."
3,Young Black Stallion,2003,Adventure,G,"$9,254,344",2,NaN
4,What's Love Got to Do With It,1993,Drama,R,"$79,618,610",1,NaN
...,...,...,...,...,...,...,...
458,Sweet Home Alabama,2002,Comedy,PG-13,"$184,521,151",1,NaN
459,Play it to the Bone,1999,Comedy,R,"$13,180,524",2,NaN
460,The Odd Life of Timothy Green,2012,Drama,PG,"$54,914,942",2,NaN
461,Honey I Blew Up the Kid,1992,Comedy,PG,"$119,162,518",2,NaN


In [21]:
print(my_data['MPAA_rating'].value_counts().sort_values(ascending=False))

PG           156
PG-13        111
R             76
G             70
Not Rated      3
Name: MPAA_rating, dtype: int64


In [22]:
top2_rating = [x for x in my_data['MPAA_rating'].value_counts().sort_values(ascending=False).head(2).index]
rating = [x for x in my_data['MPAA_rating'].value_counts().sort_values(ascending=True).head(3).index]
for l in top2_rating:
  my_data[l] = np.where(my_data['MPAA_rating']==l,1,0)

In [23]:
my_data["other_rating"] = " "
for idx,row in my_data.iterrows():
  if pd.isna(my_data.iloc[idx,3]):
    my_data["other_rating"][idx]=0
  if my_data["MPAA_rating"][idx] in rating:
    my_data["other_rating"][idx]=1
  else:
     my_data["other_rating"][idx]=0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [24]:
print(my_data['genre'].value_counts().sort_values(ascending=False))

Comedy                 146
Adventure              105
Drama                   90
Action                  30
Romantic Comedy         20
Thriller/Suspense       18
Documentary             15
Musical                 12
Western                  5
Horror                   4
Black Comedy             3
Concert/Performance      1
Name: genre, dtype: int64


In [25]:
top3_genre = [x for x in my_data['genre'].value_counts().sort_values(ascending=False).head(3).index]
genre = [x for x in my_data['genre'].value_counts().sort_values(ascending=True).head(9).index]
for l in top3_genre:
  my_data[l] = np.where(my_data['genre']==l,1,0)

In [26]:
my_data["other_genre"] = " "
for idx,row in my_data.iterrows():
  if pd.isna(my_data.iloc[idx,2]):
    my_data["other_genre"][idx]=0
  if my_data["genre"][idx] in genre:
    my_data["other_genre"][idx]=1
  else:
     my_data["other_genre"][idx]=0

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


In [27]:
my_data= my_data.drop("genre",axis = 1)

In [28]:
my_data= my_data.drop("MPAA_rating",axis = 1)

In [29]:
my_data

,movie_title,release_date,revenue,director,voice-actor,PG,PG-13,other_rating,Comedy,Adventure,Drama,other_genre
0,Recess: School's Out,2001,"$54,656,124",1,NaN,0,0,1,1,0,0,0
1,D2: The Mighty Ducks,1994,"$94,226,333",2,NaN,1,0,0,1,0,0,0
2,Home on the Range,2004,"$67,910,166",1,"[Randy Quaid, Estelle Harris, Cuba Gooding Jr....",1,0,0,1,0,0,0
3,Young Black Stallion,2003,"$9,254,344",2,NaN,0,0,1,0,1,0,0
4,What's Love Got to Do With It,1993,"$79,618,610",1,NaN,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
458,Sweet Home Alabama,2002,"$184,521,151",1,NaN,0,1,0,1,0,0,0
459,Play it to the Bone,1999,"$13,180,524",2,NaN,0,0,1,1,0,0,0
460,The Odd Life of Timothy Green,2012,"$54,914,942",2,NaN,1,0,0,0,0,1,0
461,Honey I Blew Up the Kid,1992,"$119,162,518",2,NaN,1,0,0,1,0,0,0


In [30]:
for index, row in my_data.iterrows():
  if type(my_data.iloc[index,2]) is list:
    continue;
  actors = []
  search = movie.search(row["movie_title"])
  for res in search:
    if "release_date" in res.keys():
      if str(row["release_date"]) in res.release_date:
        cast = movie.credits(res.id)["cast"]
        for x in cast:
          actors.append(x.name)
        my_data.at[index,"voice-actor"] = actors
        break;

In [31]:
my_data

,movie_title,release_date,revenue,director,voice-actor,PG,PG-13,other_rating,Comedy,Adventure,Drama,other_genre
0,Recess: School's Out,2001,"$54,656,124",1,"[Andrew Lawrence, Rickey D'Shon Collins, Pamel...",0,0,1,1,0,0,0
1,D2: The Mighty Ducks,1994,"$94,226,333",2,"[Emilio Estevez, Kathryn Erbe, Michael Tucker,...",1,0,0,1,0,0,0
2,Home on the Range,2004,"$67,910,166",1,"[Roseanne Barr, Judi Dench, Jennifer Tilly, St...",1,0,0,1,0,0,0
3,Young Black Stallion,2003,"$9,254,344",2,"[Biana Tamimi, Richard Romanus, Patrick Elias,...",0,0,1,0,1,0,0
4,What's Love Got to Do With It,1993,"$79,618,610",1,"[Angela Bassett, Laurence Fishburne, Vanessa B...",0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
458,Sweet Home Alabama,2002,"$184,521,151",1,"[Reese Witherspoon, Josh Lucas, Patrick Dempse...",0,1,0,1,0,0,0
459,Play it to the Bone,1999,"$13,180,524",2,"[Antonio Banderas, Woody Harrelson, Lolita Dav...",0,0,1,1,0,0,0
460,The Odd Life of Timothy Green,2012,"$54,914,942",2,"[Jennifer Garner, Joel Edgerton, Ron Livingsto...",1,0,0,0,0,1,0
461,Honey I Blew Up the Kid,1992,"$119,162,518",2,"[Rick Moranis, Marcia Strassman, Robert Oliver...",1,0,0,1,0,0,0


In [32]:
my_data.isnull().sum()

movie_title      0
release_date     0
revenue          0
director        23
voice-actor     23
PG               0
PG-13            0
other_rating     0
Comedy           0
Adventure        0
Drama            0
other_genre      0
dtype: int64

In [33]:
dic = dict()
for index, row in my_data.iterrows(): 
  if  type  (row['voice-actor'])is list:
   for actor in row['voice-actor']:     
     if actor in dic.keys():
       dic[actor]= dic[actor]+1
     else:
       dic[actor]=1


In [34]:
for index, row in my_data.iterrows():
  if  type  (row['voice-actor'])is list:
    fq = 0
    for actor in row['voice-actor']:
      fq = fq + dic[actor]
    my_data.at[index,"voice-actor"] = fq

In [35]:
my_data= my_data.drop("movie_title",axis = 1)

In [36]:
my_data

,release_date,revenue,director,voice-actor,PG,PG-13,other_rating,Comedy,Adventure,Drama,other_genre
0,2001,"$54,656,124",1,116,0,0,1,1,0,0,0
1,1994,"$94,226,333",2,84,1,0,0,1,0,0,0
2,2004,"$67,910,166",1,96,1,0,0,1,0,0,0
3,2003,"$9,254,344",2,7,0,0,1,0,1,0,0
4,1993,"$79,618,610",1,46,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...
458,2002,"$184,521,151",1,73,0,1,0,1,0,0,0
459,1999,"$13,180,524",2,57,0,0,1,1,0,0,0
460,2012,"$54,914,942",2,63,1,0,0,0,0,1,0
461,1992,"$119,162,518",2,26,1,0,0,1,0,0,0


In [37]:
label = my_data["revenue"]

In [38]:
label = label.str.replace('$', '')

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  """Entry point for launching an IPython kernel.


In [39]:
label

0       54,656,124 
1       94,226,333 
2       67,910,166 
3        9,254,344 
4       79,618,610 
           ...     
458    184,521,151 
459     13,180,524 
460     54,914,942 
461    119,162,518 
462    182,029,412 
Name: revenue, Length: 463, dtype: object

In [40]:
label =label.str.replace(',', '').astype(float)

In [41]:
my_data = my_data.drop("revenue",axis = 1)

In [42]:
from sklearn.impute import KNNImputer
from numpy import isnan

imputer = KNNImputer(n_neighbors=5, weights='uniform', metric='nan_euclidean')
imputer.fit(my_data)
Xtrans = imputer.transform(my_data)

In [43]:
Xtrans

array([[2.001e+03, 1.000e+00, 1.160e+02, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.994e+03, 2.000e+00, 8.400e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [2.004e+03, 1.000e+00, 9.600e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       ...,
       [2.012e+03, 2.000e+00, 6.300e+01, ..., 0.000e+00, 1.000e+00,
        0.000e+00],
       [1.992e+03, 2.000e+00, 2.600e+01, ..., 0.000e+00, 0.000e+00,
        0.000e+00],
       [1.997e+03, 6.000e+00, 1.500e+02, ..., 1.000e+00, 0.000e+00,
        0.000e+00]])

In [44]:
my_data = pd.DataFrame(Xtrans, columns=my_data.columns)

In [45]:
my_data

,release_date,director,voice-actor,PG,PG-13,other_rating,Comedy,Adventure,Drama,other_genre
0,2001.0,1.0,116.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
1,1994.0,2.0,84.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
2,2004.0,1.0,96.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0
3,2003.0,2.0,7.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
4,1993.0,1.0,46.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...
458,2002.0,1.0,73.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
459,1999.0,2.0,57.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0
460,2012.0,2.0,63.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
461,1992.0,2.0,26.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0


In [46]:
my_data.isnull().sum()

release_date    0
director        0
voice-actor     0
PG              0
PG-13           0
other_rating    0
Comedy          0
Adventure       0
Drama           0
other_genre     0
dtype: int64

In [47]:
for col in my_data.columns:
  my_data[col] = my_data[col].astype(int)

In [48]:
my_data

,release_date,director,voice-actor,PG,PG-13,other_rating,Comedy,Adventure,Drama,other_genre
0,2001,1,116,0,0,1,1,0,0,0
1,1994,2,84,1,0,0,1,0,0,0
2,2004,1,96,1,0,0,1,0,0,0
3,2003,2,7,0,0,1,0,1,0,0
4,1993,1,46,0,0,1,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...
458,2002,1,73,0,1,0,1,0,0,0
459,1999,2,57,0,0,1,1,0,0,0
460,2012,2,63,1,0,0,0,0,1,0
461,1992,2,26,1,0,0,1,0,0,0


In [49]:
x = my_data

In [50]:
x.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 463 entries, 0 to 462
Data columns (total 10 columns):
 #   Column        Non-Null Count  Dtype
---  ------        --------------  -----
 0   release_date  463 non-null    int64
 1   director      463 non-null    int64
 2   voice-actor   463 non-null    int64
 3   PG            463 non-null    int64
 4   PG-13         463 non-null    int64
 5   other_rating  463 non-null    int64
 6   Comedy        463 non-null    int64
 7   Adventure     463 non-null    int64
 8   Drama         463 non-null    int64
 9   other_genre   463 non-null    int64
dtypes: int64(10)
memory usage: 36.3 KB


In [51]:
droppedCols = list()

In [52]:
import scipy.stats 
for col_name in my_data.columns:
  corr, pval = scipy.stats.pearsonr(my_data[col_name],label)
  if pval <= 0.1:
    print(col_name)
    print(pval)
  else:
    x = x.drop(col_name, axis = 1)
    droppedCols.append(col_name)

release_date
5.462507414779024e-16
voice-actor
0.003615596539683828
other_rating
0.0106340238082625
Adventure
0.05109087764095606
other_genre
0.08888049515843961


In [53]:
droppedColsFile = open("droppedColsForRegg.pkl", "wb")
pickle.dump(droppedCols, droppedColsFile)
droppedColsFile.close()

In [54]:
X_train, X_test, y_train, y_test = train_test_split(x, label, train_size=0.80,shuffle=True, random_state=42)

In [55]:
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score

cv = KFold(n_splits = 5, random_state=1, shuffle=True)
model = linear_model.LinearRegression()
start = time.time()
#use k-fold CV to evaluate model
scores = cross_val_score(model, x, label, scoring='r2',
                         cv=cv, n_jobs=-1)
stop = time.time()
print(f"Training time: {stop - start}s")

#view mean absolute error
print("r2 score: ",np.mean(np.absolute(scores)))

scores = cross_val_score(model, x, label, scoring='neg_mean_absolute_error',
                         cv=cv, n_jobs=-1)
# dumppers('cvLinearRegressionMSE.sav',scores)
print("Mean absolute error: ",np.mean(np.absolute(scores)))

Training time: 1.2509543895721436s
r2 score:  0.5425494898899781
Mean absolute error:  130492597.60359976


In [56]:
start = time.time()
cls = linear_model.LinearRegression()
cls.fit(X_train,y_train)
dumppers('LR.sav',cls)
stop = time.time()
print(f"Training time: {stop - start}s")
prediction= cls.predict(X_test)
predictiontrain = cls.predict(X_train)


Training time: 0.005545139312744141s


In [57]:

print('Mean Square Error', metrics.mean_squared_error(y_test, prediction))
print("r2 score for the test",metrics.r2_score(y_test, prediction))
print("r2 score for the train",metrics.r2_score(y_train, predictiontrain))

Mean Square Error 3.8804902124507944e+16
r2 score for the test -0.2515853426387846
r2 score for the train 0.2497063829469115


In [58]:
from sklearn import metrics
poly_features = PolynomialFeatures(degree = 4)
# transforms the existing features to higher degree features.
X_train_poly = poly_features.fit_transform(X_train)
dumppers('polyFeatures.sav',poly_features)
# fit the transformed features to Linear Regression
poly_model = linear_model.Lasso()
start = time.time()
poly_model.fit(X_train_poly, y_train)
dumppers('poly.sav',poly_model)

stop = time.time()
print(f"Training time: {stop - start}s")

# predicting on training data-set
y_train_predicted = poly_model.predict(X_train_poly)
ypred=poly_model.predict(poly_features.transform(X_test))

print('Mean Square Error test', metrics.mean_squared_error(y_test, ypred))
print('Mean Square Error train', metrics.mean_squared_error(y_train, y_train_predicted))
print("r2 score for the test",metrics.r2_score(y_test, ypred))
print("r2 score for the train",metrics.r2_score(y_train, y_train_predicted))

Training time: 0.0390474796295166s
Mean Square Error test 3.3577421195400496e+16
Mean Square Error train 4.1271276055887096e+16
r2 score for the test -0.08298194070770393
r2 score for the train 0.6031991554718205


/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_coordinate_descent.py:648: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.635e+18, tolerance: 3.848e+15
  coef_, l1_reg, l2_reg, X, y, max_iter, tol, rng, random, positive
